## Deliverable 2. Create a Customer Travel Destinations Map.

In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os 
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 

city_data_df = pd.read_csv("WeatherPy_Vacation.csv")
city_data_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,0,Victoria,HK,84.38,overcast clouds,22.2855,114.1577,Mini Central
1,1,Jamestown,US,79.02,overcast clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown
2,2,Kapaa,US,80.58,overcast clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
3,3,Matara,LK,79.93,clear sky,5.9485,80.5353,Amaloh Boutique Resort
4,4,Palmer,US,86.50,scattered clouds,42.1584,-72.3287,Wedgewood Motel


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,0,Victoria,HK,84.38,overcast clouds,22.2855,114.1577,Mini Central
1,1,Jamestown,US,79.02,overcast clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown
2,2,Kapaa,US,80.58,overcast clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
3,3,Matara,LK,79.93,clear sky,5.9485,80.5353,Amaloh Boutique Resort
4,4,Palmer,US,86.50,scattered clouds,42.1584,-72.3287,Wedgewood Motel
5,5,Puerto Ayora,EC,78.76,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
6,6,Athens,GR,86.25,clear sky,37.9795,23.7162,Safestay Athens
7,7,Hami,CN,81.64,overcast clouds,42.8000,93.4500,Laoyutang Resort
8,8,Arraial Do Cabo,BR,78.76,clear sky,-22.9661,-42.0278,Pousada Porto Praia
9,9,Itoman,JP,78.82,mist,26.1247,127.6694,Southern Beach Hotel & Resort Okinawa


In [8]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                160
City                   160
Country                160
Max Temp               160
Current Description    160
Lat                    160
Lng                    160
Hotel Name             160
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Victoria,HK,84.38,overcast clouds,22.2855,114.1577,
1,Jamestown,US,79.02,overcast clouds,42.0970,-79.2353,
2,Kapaa,US,80.58,overcast clouds,22.0752,-159.3190,
3,Matara,LK,79.93,clear sky,5.9485,80.5353,
4,Palmer,US,86.50,scattered clouds,42.1584,-72.3287,
5,Puerto Ayora,EC,78.76,broken clouds,-0.7393,-90.3518,
6,Athens,GR,86.25,clear sky,37.9795,23.7162,
7,Hami,CN,81.64,overcast clouds,42.8000,93.4500,
8,Arraial Do Cabo,BR,78.76,clear sky,-22.9661,-42.0278,
9,Itoman,JP,78.82,mist,26.1247,127.6694,


In [11]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"        
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
hotel_df
        

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Victoria,HK,84.38,overcast clouds,22.2855,114.1577,Mini Central
1,Jamestown,US,79.02,overcast clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown
2,Kapaa,US,80.58,overcast clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
3,Matara,LK,79.93,clear sky,5.9485,80.5353,Amaloh Boutique Resort
4,Palmer,US,86.50,scattered clouds,42.1584,-72.3287,Wedgewood Motel
...,...,...,...,...,...,...,...
155,Ramgarh,IN,82.83,overcast clouds,23.6333,85.5167,HOTEL TREAT RESIDENCY
156,Frutal,BR,87.46,clear sky,-20.0247,-48.9406,Hotel Recanto Dos Ipês
157,Umm Lajj,SA,85.77,clear sky,25.0213,37.2685,Moon light Furnished Units
158,Alibag,IN,82.47,overcast clouds,18.6411,72.8792,"Radisson Blu Resort & Spa - Alibaug, India"


In [12]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [13]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))